                            Import Libraries

In [1]:
from bs4 import BeautifulSoup as bs
import requests

In [2]:
def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace('\xa0',' ') for li in row_data.find_all('li')]
    else:
        return row_data.get_text(" ", strip=True).replace('\xa0',' ')

In [3]:
#clean data to remove [1] [2], date format eg (09-25-1953)
def clean_tags(soup):
    for tag in soup.find_all(["sup","span"]):
        tag.decompose()

In [4]:
def get_info_box(url):

    r =  requests.get(url)
    soup = bs(r.content)


    info_box = soup.find(class_='infobox vevent')
    info_row = info_box.find_all("tr")

    clean_tags(soup)

    movie_info ={}
    for index,row in enumerate(info_row):
        if index==0:
            movie_info['Title'] = row.find("th").get_text(" ", strip=True)
        elif index==1:
            continue
        else:
            contenet_key = row.find("th").get_text(" ", strip=True)
            contenet_value = get_content_value(row.find("td"))
            movie_info[contenet_key] = contenet_value

    return movie_info

In [5]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')
soup = bs(r.content)
contenets = soup.prettify()

In [6]:
movies = soup.select('.wikitable.sortable i')
movie_info_list =[]
base_url = 'https://en.wikipedia.org'

for index,movies in enumerate(movies):
    if index==2:
        break 
    try:
        relative_path = movies.a['href']
        full_path = base_url+relative_path
        title = movies.a['title']

        movie_info_list.append(get_info_box(full_path))
        
        # print(relative_path)
        # print(title)
        # print()
        
    except Exception as e:
        print(movies.get_text(), e)

In [7]:
# movie_info_list

                                                            Save Data to JSON

In [8]:
import json

In [9]:
def save_data(title,data):
    with open(title,'w', encoding='utf-8') as f:
        json.dump(data,f,ensure_ascii=False, indent=2)

In [10]:
def load_data(title):
    with open(title,encoding='utf-8') as f:
        return json.load(f)

In [11]:
save_data('Disney Dataset',movie_info_list)